In [115]:
!az login

[

In [125]:
subscription_id = !az account show --query id --output tsv
subscription_id = subscription_id[0]

In [126]:
# Get the resource group name and workspace name for the workspace from the config.conf file using pyhocon
from pyhocon import ConfigFactory
config = ConfigFactory.parse_file("config.conf")
resource_group = config.get_string("RESOURCE_GROUP")
workspace_name = config.get_string("WORKSPACE_NAME")

In [127]:

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

# NOTE: It's very import to always set the resource_group and workspace_name when creating the MLClient
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)
print(ml_client)

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x000001DE4C367EB0>,
         subscription_id=ea54fca2-3a3e-4b3b-91e7-a7bf971e0443,
         resource_group_name=rg-nano-gpt,
         workspace_name=ml-nano-gpt)


## Training

In [108]:
from azure.ai.ml.entities import Environment

custom_env = Environment(
    name='nano_gpt_env',
    description="Custom environment for nano gpt training",
    conda_file="../environment.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.8-cudnn8-ubuntu22.04"
)
ml_client.environments.create_or_update(custom_env)

Environment({'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'nano_gpt_env', 'description': 'Custom environment for nano gpt training', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': True, 'id': '/subscriptions/ea54fca2-3a3e-4b3b-91e7-a7bf971e0443/resourceGroups/rg-nano-gpt/providers/Microsoft.MachineLearningServices/workspaces/ml-nano-gpt/environments/nano_gpt_env/versions/3', 'Resource__source_path': None, 'base_path': 'c:\\Users\\d92343\\Documents\\IT\\Projects\\Private\\nanoGPTExperiments\\azure_deployment', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x000001DE4C3C7EE0>, 'serialize': <msrest.serialization.Serializer object at 0x000001DE4C3F0AF0>, 'version': '3', 'latest_version': None, 'conda_file': {'channels': ['nvidia/label/cuda-11.8.0', 'pytorch', 'conda-forge', 'defaults'], 'dependencies': ['python=3.10', 'pip', {'pip': ['numpy', 'transformers'

### Shakespeare Config

In [140]:
# Load params from src/config.json
import json
with open("../src/configs/config.json") as f:
    params = json.load(f)['train_params']




Before running the job, remove all unnecessary files from the src folder

In [141]:
from azure.ai.ml import command

# configure job
job = command(
    code="../src",
    command="python pipeline.py",
    environment=custom_env,
    compute="aml-cluster-nano-gpt",
    display_name=params["experiment_name"],
    experiment_name=params["experiment_name"],
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Uploading src (27.77 MBs): 100%|##########| 27773665/27773665 [00:08<00:00, 3458944.19it/s]




Monitor your job at https://ml.azure.com/runs/mighty_leg_zztz0ny1nt?wsid=/subscriptions/ea54fca2-3a3e-4b3b-91e7-a7bf971e0443/resourcegroups/rg-nano-gpt/workspaces/ml-nano-gpt&tid=1fdb6d07-480b-4c31-a913-e847d51e7446


In [94]:
returned_job

Experiment,Name,Type,Status,Details Page
nano-gpt-training,heroic_spinach_qslcj4s1d6,command,Starting,Link to Azure Machine Learning studio
